# Kryptologie Lab - Übung 04

In [179]:
import numpy as np

In [180]:
encryptGaloisMatrix = np.array([[2,3,1,1],[1,2,3,1],[1,1,2,3],[3,1,1,2]])
decryptGaloisMatrix = np.array([[0xE,0xB,0xD,9],[9,0xE,0xB,0xD],[0xD,9,0xE,0xB],[0xB,0xD,9,0xE]])

In [181]:
with open('SBox.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))

with open('SBoxInvers.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox_inverse = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))

## Init Galoiskörper-Operationen

In [182]:
def gfAddition(a,b):
    return np.bitwise_xor(a,b)

def xTimes(a):
    t = a << 1
    if a & (1 << 7) != 0:
        t = t ^ 0x1b
    return t

def gfMultiply(a,b):
    index = 0
    out = 0
    while b != 0:
        if (b & 1 == 1):
            val = a
            for i in range(index):
                val = xTimes(val)
            out = gfAddition(out, val)
        index += 1
        b = b >> 1
    return out

## Init algorithm steps

In [183]:
def initBlock(textblock):
    chars = [ord(c) for c in textblock]
    if (len(chars) < 16):
        fill = [0 for i in range(16 - len(chars))]
        chars.extend(fill)
    return np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4), order="F")

def blockToText(textblock):
    text = textblock.transpose().flatten()
    toChar = lambda c: chr(c)
    return "".join(np.array([toChar(c) for c in text]))

def addRoundKey(textblock, key):
    return np.bitwise_xor(textblock, key)
    
def subByte(byte, box):
    row = (byte >> 4) & 0xF
    col = byte & 0xF
    return box[row, col]

def subBytes(textblock, reverse=False):
    if reverse:
        box = sbox_inverse
    else:
        box = sbox
    for y in range(len(textblock)):
        for x in range(len(textblock[y])):
            textblock[y,x] = subByte(textblock[y,x], box)
    return textblock

def shiftRows(textblock, reverse=False):
    for i in range(4):
        if reverse:
            shiftAmount = i
        else:
            shiftAmount = -i
        textblock[i] = np.roll(textblock[i], shiftAmount)
    return textblock

def subColumn(inputColumn, matrix):
    outputColumn = np.copy(inputColumn)
    for row in range(4):
        val = 0
        for column in range(4):
            val = gfAddition(val, gfMultiply(matrix[row, column], inputColumn[column]))
        outputColumn[row] = val
    return outputColumn

def mixColumns(textblock, reverse=False):
    if reverse:
        matrix = decryptGaloisMatrix
    else:
        matrix = encryptGaloisMatrix
    for column in range(4):
        textblock[:, column] = subColumn(textblock[:, column], matrix)
    return textblock

## Init algorithm process

In [184]:
def encryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[0])
    for i in range(1, 10):
        textblock = subBytes(textblock)
        textblock = shiftRows(textblock)
        textblock = mixColumns(textblock)
        textblock = addRoundKey(textblock, keys[i])
    textblock = subBytes(textblock)
    textblock = shiftRows(textblock)
    textblock = addRoundKey(textblock, keys[10])
    return blockToText(textblock)

def decryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[10])
    textblock = shiftRows(textblock, reverse=True)
    textblock = subBytes(textblock, reverse=True)
    for i in range(9, 0, -1):
        textblock = addRoundKey(textblock, keys[i])
        textblock = mixColumns(textblock, reverse=True)
        textblock = shiftRows(textblock, reverse=True)
        textblock = subBytes(textblock, reverse=True)
    textblock = addRoundKey(textblock, keys[0])
    return blockToText(textblock)

## Init block splitting

In [185]:
def encrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([encryptBlock(block, keys) for block in blocks])

def decrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([decryptBlock(block, keys) for block in blocks])

## Schlüsselgenerierung

In [186]:
key = ["EINSCHLUESSEL123" for i in range(11)]

In [187]:
def rotWord(word):
    pass

def subWord(word):
    return np.roll([subByte(byte, sbox) for byte in word], shift=-1)

def rcon(i):
    array = [0x1, 0x2, 0x4, 0x8, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36]
    return array[i]

# input 16 character string
def generateKey(key):
    chars = [ord(c) for c in key]
    words = []
    words.append(chars[0:4])
    words.append(chars[4:8])
    words.append(chars[8:12])
    words.append(chars[12:16])
    for i in range(4, 44):
        if i % 4 == 0:
            val = gfAddition(words[i-4], rcon(round(i/4) - 1))
            val = gfAddition(subWord(words[i-1]), val)
            words.append(val)
        else:
            words.append(gfAddition(words[i-4], words[i-1]))
    keys = np.empty((11, 4,4), dtype=np.dtype('B'))
    for i in range(0, 44, 4):
        keyNumber = round(i/4)
        keys[keyNumber, 0] = words[i]
        keys[keyNumber, 1] = words[i+1]
        keys[keyNumber, 2] = words[i+2]
        keys[keyNumber, 3] = words[i+3]
    return keys

In [191]:
keys = generateKey("EINSCHLUESSEL123")
print(keys)
print(keys.shape)

[[[ 69  73  78  83]
  [ 67  72  76  85]
  [ 69  83  83  69]
  [ 76  49  50  51]]

 [[131 107 140 123]
  [192  35 192  46]
  [133 112 147 107]
  [201  65 161  88]]

 [[  2  91 228 164]
  [194 120  36 138]
  [ 71   8 183 225]
  [142  73  22 185]]

 [[ 61  24 182 185]
  [255  96 146  51]
  [184 104  37 210]
  [ 54  33  51 107]]

 [[200 211 193 180]
  [ 55 179  83 135]
  [143 219 118  85]
  [185 250  69  62]]

 [[245 173  99 242]
  [194  30  48 117]
  [ 77 197  70  32]
  [244  63   3  30]]

 [[160 246  49 109]
  [ 98 232   1  24]
  [ 47  45  71  56]
  [219  18  68  38]]

 [[ 41 173 134 148]
  [ 75  69 135 140]
  [100 104 192 180]
  [191 122 132 146]]

 [[115 114  73  28]
  [ 56  55 206 144]
  [ 92  95  14  36]
  [227  37 138 182]]

 [[ 87  23  28  22]
  [111  32 210 134]
  [ 51 127 220 162]
  [208  90  86  20]]

 [[223 144 208  80]
  [176 176   2 214]
  [131 207 222 116]
  [ 83 149 136  96]]]
(11, 4, 4)


In [189]:
text = "Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger."
print("Input: ", text)
cipher = encrypt(text, keys)
print("cypher: ", cipher)
new_text = decrypt(cipher, keys)
print("text: ", new_text)

Input:  Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger.
cypher:  7RÉG×&@çÜc.kÚÍ@¯Ïý¬UGèâ2}ïÁÅºÃ¡;V/%y (äÈXÝû&uÐß|W!hE­mD%ÖG×Ý °ÎÒú¬=ü¹Wo®äÛY
Lµã²3PÀäT
text:  Hallo, das ist ein geheimer Text. Mal schauen, ob er geheim bleibt. Der Text ist noch ein Stueck laenger.
